In [ ]:
# !pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
#Load libraries
import time
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision import models
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import matplotlib.pyplot as plt

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Augmentation object creation

In [3]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

## Data loading

In [ ]:
#check image
#Path for training and testing directory
train_path='C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset_pt/train'
test_path='C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset_pt/test'

train_img =  torchvision.datasets.ImageFolder(root = train_path,transform=transformer)
test_img = torchvision.datasets.ImageFolder(root = test_path,transform=transformer)

In [ ]:
def show_transformed_image(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size = 6, shuffle=True)
    batch = next(iter(loader))
    images, labels = batch
    
    grid = torchvision.utils.make_grid(images, nrow = 3)
    plt.figure(figsize = (11,11))
    plt.imshow(np.transpose(grid, (1,2,0)))
    print('Labels:', labels)

In [ ]:
show_transformed_image(train_img)

In [4]:
#Dataloader

#Path for training and testing directory
train_path='C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset_pt/train'
test_path='C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset_pt/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

# train_loader=DataLoader(
#     train_img,
#     batch_size=64, shuffle=True
# )
# test_loader=DataLoader(
#     test_img,
#     batch_size=32, shuffle=True
# )

In [5]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [6]:
print(classes)

['Fruit', 'Packages', 'Vegetables']


## Build model

In [7]:
# model=ConvNet(num_classes=len(classes)).to(device)
model = models.resnet50()
model.load_state_dict(torch.load('resnet50-0676ba61.pth'))

for parameter in model.parameters():
    parameter.requires_grad = False
    
model.fc = torch.nn.Linear(in_features=2048,out_features=len(classes),bias=True)

model = model.to(device)

In [8]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()
num_epochs=10

In [9]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [10]:
print(train_count,test_count)

1746 165


In [11]:
model.train()
start = time.time()
for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
end = time.time()
print("Time taken:", end - start)

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 Train Loss: tensor(0.5906) Train Accuracy: 0.761168384879725 Test Accuracy: 0.8181818181818182
Epoch: 1 Train Loss: tensor(0.3103) Train Accuracy: 0.8900343642611683 Test Accuracy: 0.9212121212121213
Epoch: 2 Train Loss: tensor(0.2259) Train Accuracy: 0.93184421534937 Test Accuracy: 0.9212121212121213
Epoch: 3 Train Loss: tensor(0.2047) Train Accuracy: 0.936426116838488 Test Accuracy: 0.9212121212121213
Epoch: 4 Train Loss: tensor(0.1949) Train Accuracy: 0.9312714776632303 Test Accuracy: 0.9393939393939394
Epoch: 5 Train Loss: tensor(0.1833) Train Accuracy: 0.9375715922107675 Test Accuracy: 0.9575757575757575
Epoch: 6 Train Loss: tensor(0.1500) Train Accuracy: 0.9564719358533792 Test Accuracy: 0.9333333333333333
Epoch: 7 Train Loss: tensor(0.1389) Train Accuracy: 0.9587628865979382 Test Accuracy: 0.9454545454545454
Epoch: 8 Train Loss: tensor(0.1333) Train Accuracy: 0.9639175257731959 Test Accuracy: 0.9454545454545454
Epoch: 9 Train Loss: tensor(0.1420) Train Accuracy: 0.95246